In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import urllib
import cv2 as cv
import plotly.express as px
from scipy import stats
from statsmodels.stats.weightstats import ztest
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

In [3]:
airbnb_data = pd.read_csv('/kaggle/input/new-york-city-airbnb-open-data/AB_NYC_2019.csv')

In [4]:
airbnb_data.head()

## Data Wrangling

In [5]:
airbnb_data.info()

In [6]:
airbnb_data.isnull().sum()

Comapring the number of missing values to the number of non-missing values, missing values are quite a few in *last_review*, and *reviews_per_month* columns. We can inspect the *last_review* column in order to see the relationship between rows that contain missing values (aka NaNs).

In [7]:
airbnb_data[pd.isnull(airbnb_data['last_review'])]


The same rows in last_review and review_per_month columns are missing values. Also for these rows, *number_of_reviews* is 0. Hence, we can replace *number_of_reviews* with 0.

In [8]:
airbnb_data.fillna({'reviews_per_month':0}, inplace=True)

As we see, *name*, *host_name*, *last_review*, and *reviews_per_month* columns contain missing values. We can drop the rows that contain missing values in *name* and *host_name* columns since the number of missing values are not high in those columns. Besides, as our further work is not related with *name* and *host_name* columns, we can drop the both columns. Having no information about the review for this dataset, we can also drop the *last_review* column.

In [9]:
airbnb_data.drop(['name','host_name','last_review'], axis=1, inplace=True)

We can check our data info for datatypes again.

In [10]:
airbnb_data.info()

All data types seems to be good and there are no more missing values.


In [11]:
airbnb_data.head()

## Data Exploration and Visualization

### Host_id Column
We can start with exploring the dataset. As each *host_id* represents the number of visits of a host, we can visualize it using the bar graph for teh first 10 entries of this dataset.

In [12]:
host_count=airbnb_data.host_id.value_counts().head(10)
host_count

In [13]:
sns.set(rc={'figure.figsize':(12,8)})
visual1=host_count.plot(kind='bar', color=['C0','C1','C2','C3','C4','C5'])
visual1.set_ylabel('Counts')
visual1.set_xlabel('Host ID')
visual1.set_title('Most 10 hosts in NYC ')
visual1.set_xticklabels(visual1.get_xticklabels(), rotation=45)
plt.show()

### neighbourhood_group Column

In [14]:
neighborhood_group_count = airbnb_data.neighbourhood_group.value_counts()
neighborhood_group_count

We observe that Manhattan hosts the highest number of Airbnbs in NY.

In [55]:
sns.set(rc={'figure.figsize':(12,8)})
visual2 = neighborhood_group_count.plot(kind='bar')
visual2.set_ylabel('Counts')
visual2.set_xlabel('Neighborhood Groups')
visual2.set_title('Most 10 hosts in NYC ')
visual2.set_xticklabels(visual2.get_xticklabels(), rotation=45)
plt.show()

## Price Distribution with respect to Neighborhood Groups

In [16]:
sns.set(rc={'figure.figsize':(12,8),"axes.titlesize":24,"axes.labelsize":18})
df1 =  airbnb_data[airbnb_data['price']<1000]
sns.boxplot(data=df1,x='neighbourhood_group',y='price').set_title('Price distribution respect to neighbourhood group')
plt.show()

In [17]:
sns.distplot(df1['price'])
plt.title('Price Distribution')
plt.show()

We notice that Manhattan has the greatest price range for listings, with an average of \$150 per night, followed by Brooklyn at $90 per night. The allocations in Queens and Staten Island appear to be relatively comparable, with the Bronx being the cheapest of the three. This price distribution and density was quite predictable; for example, it is no secret that Manhattan is one of the most expensive cities in the world to live in, but the Bronx looks to have lower living standards.

## Neighbourhood

As we saw earlier with the unique values for neighborhoods, there are far too many to focus on; therefore, we are going to focus on the top ten neighborhoods with the most listings.

In [18]:
n_count=airbnb_data.neighbourhood.value_counts()
n_count.head(10)

In [19]:
region=airbnb_data.groupby(['neighbourhood_group','neighbourhood']).count()[['id']]
region=region['id'].groupby(level=0, group_keys=False)
region.nlargest()

## Relationship between Room Types and Neighbourhood

Now let's combine this with our boroughs and room types to create a rich visualization. A catplot would be a better choice because we are interested in multiple attributes together and a count.

In [20]:
neighborhood_group=airbnb_data.loc[airbnb_data['neighbourhood'].isin(['Williamsburg','Bedford-Stuyvesant','Harlem','Bushwick',
                    'Upper West Side','Hell\'s Kitchen','East Village','Upper East Side','Crown Heights','Midtown'])]
viasul2=sns.catplot(x='neighbourhood', hue='neighbourhood_group', col='room_type', data=neighborhood_group, kind='count')
viasul2.set_xticklabels(rotation=60)

The most striking observation is that 'Shared room' type Airbnb listings are scarcely available in the ten most densely populated neighborhoods. Then we can see that only two boroughs are represented for these ten neighborhoods: Manhattan and Brooklyn. This is to be expected, as Manhattan and Brooklyn are two of the most visited destinations and thus have the most listing availability.

## Price Distribution of Room Types

In [21]:
sns.set(rc={'figure.figsize':(10,8),"axes.titlesize":24,"axes.labelsize":18})

df2 = airbnb_data[airbnb_data.price<600]

visual3 = sns.violinplot(data=df2, x='room_type', y='price')

visual3.set_title('Distribution of prices for each room type')

plt.show()

# Details of the Room

In [22]:
airbnb_data.room_type.value_counts()

In [23]:
room_labels=list(airbnb_data["room_type"].unique())
room_labels

In [24]:
room_vals=list(airbnb_data.room_type.value_counts())
room_vals

In [25]:
fig = plt.figure()
fig.set_figheight(9)
fig.set_figwidth(16)
plt.subplot(1, 2, 1)
plt.axis('equal')
plt.pie(room_vals,labels=room_labels,radius=1.5,autopct='%0.2f%%',textprops={'fontsize': 20})
plt.subplot(1, 2, 2)
plt.hist(x=airbnb_data['room_type'])
plt.xlabel('Room type',fontsize=25)
plt.ylabel('Count',fontsize=25)
plt.xticks(fontsize=20, rotation=30)
plt.yticks(fontsize=20)
fig.tight_layout(pad=2.6)
fig.suptitle('Distribution of room type',fontsize=30)
plt.show()

## Room Types and Neighbourhood Group

In [26]:
roomtype=airbnb_data[['room_type','neighbourhood_group']]
roomtype

In [27]:
visual2=sns.countplot(x='neighbourhood_group', hue='room_type', data=airbnb_data)
visual2.set_title('Relationship between Room Types and Neighbourhood Group',fontsize=24)
visual2.set(xlabel='Boroughs ')
plt.show()

## Distributions in Maps

In [28]:
#initializing the figure size
plt.figure(figsize=(10,8))

#loading the png NYC image found on Google and saving to my local folder along with the project
NewYorkCity_image_path = '/kaggle/input/new-york-city-airbnb-open-data/New_York_City_.png'
NewYorkCity_image = cv.imread(NewYorkCity_image_path)

#scaling the image based on the latitude and longitude max and mins for proper output
plt.imshow(NewYorkCity_image, zorder=0, extent=[-74.258, -73.7, 40.49,40.92])

ax=plt.gca()

df3=airbnb_data[airbnb_data.price < 500]

df3.plot(kind='scatter', x='longitude', y='latitude', label='availability_365', c='price', ax=ax, 
           cmap=plt.get_cmap('jet'), colorbar=True, alpha=0.4, zorder=5)

plt.title('Price Distribution on the map',fontsize=24)

plt.legend()

plt.show()

In [29]:
fig= px.scatter_mapbox(airbnb_data,lat="latitude", lon="longitude",
                     color="neighbourhood_group", 
                     hover_name="id",hover_data=["neighbourhood"], 
                     zoom=9, height=500)

fig.update_layout(mapbox_style="open-street-map", margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

## Minimum Number of Nights

In [30]:
airbnb_data.minimum_nights.value_counts().head(10)

In [31]:
visual4=sns.distplot(airbnb_data['minimum_nights'], rug=False, kde=False, color="red")
visual4.set_yscale('log')
visual4.set_xlabel('minimum nights')
visual4.set_ylabel('count')

## Distribution of Availability in a Year

In [32]:
airbnb_data.availability_365.value_counts().head()

In [33]:
visual5=sns.distplot(airbnb_data['availability_365'], rug=False, kde=False, color='purple')
visual5.set_title('Distribution of Availability')
plt.show()

## Statistics

### t-Test for mean of prices (Manhattan and Brooklyn)

+ H0 = The null hypothesis in these data sets will be that both the data sets are significantly similar.
+ H1 = The alternative hypothesis will be that both the data sets are significantly different.

In [34]:
price_Manhattan=df1[df1['neighbourhood_group']=='Manhattan']['price']
price_Brooklyn=df1[df1['neighbourhood_group']=='Brooklyn']['price']

tTest_price_M_B=stats.ttest_ind(price_Manhattan,price_Brooklyn,equal_var=False)
tTest_price_M_B

If we set the significance level at 5%, our p-value must be higher than the specified significance level in order to reject the null hypothesis. Because the p-value=0 in our case is less than 0.05 (5 percent), we disregard the null hypothesis. The t-test accurately reveals that the means of both datasets differ and are statistically significant from one another.

## Z-Test for mean of prices (Queens and Staten Island)

+ H0 = The null hypothesis in these data sets will be that both the data sets are significantly similar.
+ H1 = The alternative hypothesis will be that both the data sets are significantly different.

In [35]:
price_Queens = df1[df1['neighbourhood_group']=='Queens']['price']
price_Staten = df1[df1['neighbourhood_group']=='Staten Island']['price']

ztest(price_Queens, price_Staten,value=0 )

If we set the significance level at 5%, our p-value must be higher than the chosen significance level in order to reject the null hypothesis. We accept the null hypothesis because the p-value=0.67 is greater than 0.05 (5 percent) in our example. The Z-Test correctly indicates that the means of both datasets are statistically insignificantly different.

## Z-Test for the average of prices (Private room and Shared room)

In [36]:
price_private=df2[df2['room_type']=='Private room']['price']
price_shared=df2[df2['room_type']=='Shared room']['price']

ztest(price_private, price_shared)

If we set the significance level at 5%, our p-value must be higher than the chosen significance level in order to reject the null hypothesis. Because the p-value=0 in our example is less than 0.05 (5 percent), we disregard the null hypothesis. The Z-Test correctly indicates that the means of both datasets differ and are statistically significant from one another.

## Correlation between Number of Reviews and Price

In [37]:
# Make a scatter plot
plt.plot(df1['number_of_reviews'],df1['price'],marker='.',linestyle='none')
# Label the axes
plt.xlabel('reviews')
plt.ylabel('price')
#show the result
plt.show()

In [38]:
def pearson_r(x, y):
    
    corr_mat = np.corrcoef(x,y)
    
    return corr_mat[0,1]

r = pearson_r(df1['number_of_reviews'],df1['price'])

print(r)

It is clear that the two variables have no correlation.

In [39]:
sns.heatmap(df1.corr(), square=True, cmap='RdYlGn',annot = True)

## Modelling

In [40]:
stats.tmax(airbnb_data['price'])

firstquartile = np.percentile(airbnb_data.price, 25)

thirdquartile = np.percentile(airbnb_data.price, 75)

upperlim = thirdquartile + (thirdquartile-firstquartile) * 1.5

print(upperlim)

Let's eliminate outliers.

In [41]:
df334 = airbnb_data[airbnb_data.price<334]
airbnb_data_without_outliers = pd.get_dummies(df334)
airbnb_data_without_outliers.head()

In [42]:
airbnb_data_without_outliers.shape

### Linear Regression

In [43]:
feature_cols1 = airbnb_data_without_outliers.drop(['price'],axis=1)

X = feature_cols1

y = airbnb_data_without_outliers['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

reg = LinearRegression()

reg.fit(X_train,y_train)

y_pred = reg.predict(X_test)

print("R^2: {}".format(reg.score(X_test, y_test)))

rmse = np.sqrt(metrics.mean_squared_error(y_test,y_pred))

print("Root Mean Squared Error: {}".format(rmse))
cv_scores = cross_val_score(reg,X,y,cv=5)

# Print the 5-fold cross-validation scores
print("CV Scores: {}".format(cv_scores))
print("Average 5-Fold CV Score: {}".format(np.mean(cv_scores)))

The model's score has improved as a result of using all features. This makes sense because the model has more data from which to learn.

## Lasso Regression

In [44]:
#default regularization parameter value=1

lasso = Lasso()
lasso.fit(X_train,y_train)
lasso_pred = lasso.predict(X_test)

train_score = lasso.score(X_train,y_train)
cv_score = cross_val_score(lasso,X,y,cv=5)
test_score = lasso.score(X_test,y_test)
coeff_used = np.sum(lasso.coef_!=0)

print ("training score:", train_score)
print ("test score: ", test_score)
print("Average 5-Fold CV Score: {}".format(np.mean(cv_score)))
print ("number of features used: ", coeff_used,"\n")

In [45]:
lasso01 = Lasso(alpha=0.1, max_iter=10e5) # regularization parameter value alpha = 0.1
lasso01.fit(X_train,y_train)

train_score01=lasso01.score(X_train,y_train)
test_score01=lasso01.score(X_test,y_test)
coeff_used01 = np.sum(lasso01.coef_!=0)
cv_score01=cross_val_score(lasso01,X,y,cv=5)

print ("training score for alpha=0.1:", train_score01)
print ("test score for alpha=0.1: ", test_score01)
print("Average 5-Fold CV Score: {}".format(np.mean(cv_score01)))
print ("number of features used for alpha=0.1: ", coeff_used01)

In [46]:
lasso001 = Lasso(alpha=0.01, max_iter=10e5) # regularization parameter value alpha = 0.01
lasso001.fit(X_train,y_train)

train_score001=lasso001.score(X_train,y_train)
test_score001=lasso001.score(X_test,y_test)
coeff_used001 = np.sum(lasso001.coef_!=0)
cv_score001=cross_val_score(lasso001,X,y,cv=5)

print ("training score for alpha=0.01:", train_score001)
print ("test score for alpha=0.01: ", test_score001)
print("Average 5-Fold CV Score: {}".format(np.mean(cv_score001)))
print ("number of features used for alpha=0.01: ", coeff_used001)

In [47]:
lasso0001 = Lasso(alpha=0.001, max_iter=10e5) # regularization parameter value alpha = 0.001
lasso0001.fit(X_train,y_train)

train_score0001 = lasso0001.score(X_train,y_train)
test_score0001 = lasso0001.score(X_test,y_test)
coeff_used0001 = np.sum(lasso01.coef_!=0)
cv_score0001 = cross_val_score(lasso0001,X,y,cv=5)

print ("training score for alpha=0.1:", train_score0001)
print ("test score for alpha=0.1: ", test_score0001)
print("Average 5-Fold CV Score: {}".format(np.mean(cv_score0001)))
print ("number of features used for alpha=0.001: ", coeff_used0001)

#### Linear Regression

In [48]:
lr = LinearRegression()
lr.fit(X_train,y_train)
lr_train_score=lr.score(X_train,y_train)
lr_test_score=lr.score(X_test,y_test)
print ("LR training score:", lr_train_score)
print ("LR test score: ", lr_test_score)

In [49]:
plt.subplot(2,1,1)
plt.plot(lasso.coef_,alpha=0.7,linestyle='none',marker='*',markersize=5,color='red',label=r'Lasso; $\alpha = 1$',zorder=7) # alpha here is for transparency
plt.plot(lasso01.coef_,alpha=0.5,linestyle='none',marker='d',markersize=6,color='blue',label=r'Lasso; $\alpha = 0.1$') # alpha here is for transparency

plt.xlabel('Coefficient Index',fontsize=16)
plt.ylabel('Coefficient Magnitude',fontsize=16)
plt.legend(fontsize=13,loc=4)

plt.subplot(2,1,2)
plt.plot(lasso.coef_,alpha=0.7,linestyle='none',marker='*',markersize=5,color='red',label=r'Lasso; $\alpha = 1$',zorder=7) # alpha here is for transparency
plt.plot(lasso01.coef_,alpha=0.5,linestyle='none',marker='d',markersize=6,color='blue',label=r'Lasso; $\alpha = 0.1$') # alpha here is for transparency
plt.plot(lasso001.coef_,alpha=0.8,linestyle='none',marker='v',markersize=6,color='black',label=r'Lasso; $\alpha = 0.01$') # alpha here is for transparency
plt.plot(lasso0001.coef_,alpha=0.8,linestyle='none',marker='s',markersize=6,color='yellow',label=r'Lasso; $\alpha = 0.001$') # alpha here is for transparency
plt.plot(lr.coef_,alpha=0.7,linestyle='none',marker='o',markersize=5,color='green',label='Linear Regression',zorder=2)

plt.xlabel('Coefficient Index',fontsize=16)
plt.ylabel('Coefficient Magnitude',fontsize=16)
plt.legend(fontsize=13,loc=4)
plt.tight_layout()
plt.show()

Let’s understand the plot and the code in a short summary.

+ The default value of regularization parameter in Lasso regression (given by $\alpha$ ) is 1. With this, out of 237 features in cancer data-set, only 11 features are used (non zero value of the coefficient).

+ When $\alpha$ = 0.1, non-zero features = 44, training and test score increases.

#Comparison of coefficient magnitude for two different values of alpha are shown in the left panel of figure 2. For $\alpha$ = 1, we can see most of the coefficients are zero or nearly zero, which is not the case for alpha=0.1.

#Further reduce $\alpha$ = 0.001, non-zero features = 211. Training and test scores are similar to basic linear regression case.

### Best Parameter in Lasso Regression


In [50]:
normalizes= ([True,False])
alphas = np.array([1,0.1,0.01,0.001,0])

# Instantiate a lasso regression classifier
lasso = Lasso()

# Instantiate the GridSearchCV object
lasso_cv = GridSearchCV(lasso, param_grid=(dict(alpha=alphas, normalize= normalizes)),scoring='r2',cv=5)

# Fit it to the data
lasso_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Tuned Lasso Regression Parameters: {}".format(lasso_cv.best_params_)) 
print("Best R^2 is {}".format(lasso_cv.best_score_))

lasso_pred= lasso_cv.predict(X_test)
rmse_lasso=np.sqrt(metrics.mean_squared_error(y_test,lasso_pred))
print("Root Mean Squared Error: {}".format(rmse_lasso))

When we use GridSearchCV for best parameters in Lasso Regression, we see that we got best $R^2$ result with $\alpha$ = 0.001.



### Ridge Regression

In [51]:
# Setup the hyperparameter grid
normalizes= ([True,False])
alphas = np.array([1,0.1,0.01,0.001,0])
# Instantiate a lasso regression
ridge = Ridge()

# Instantiate the GridSearchCV object
ridge_cv = GridSearchCV(ridge, param_grid=(dict(alpha=alphas, normalize= normalizes)),scoring='r2', cv=5)

# Fit it to the data
ridge_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Tuned Ridge Regression Parameters: {}".format(ridge_cv.best_params_)) 
print("Best R^2 is {}".format(ridge_cv.best_score_))

ridge_pred= ridge_cv.predict(X_test)
rmse_ridge=np.sqrt(metrics.mean_squared_error(y_test,ridge_pred))
print("Root Mean Squared Error: {}".format(rmse_ridge))

When we use gridsearchcv for best parameters in Ridge Regression, we see that we got best $R^2$ result with $\alpha$ = 1.

### ElasticNet Regression

In [52]:
# Setup the hyperparameter grid
normalizes= ([True,False])
alphas = np.array([1,0.1,0.01,0.001,0])

# Instantiate a ElasticNet regression
elasticreg= ElasticNet()

# Instantiate the GridSearchCV object
elasticreg_cv = GridSearchCV(elasticreg, param_grid=(dict(alpha=alphas, normalize= normalizes)),scoring='r2', cv=5)

# Fit it to the data
elasticreg_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Tuned ElasticNet Regression Parameters: {}".format(elasticreg_cv.best_params_)) 
print("Best R^2 is {}".format(elasticreg_cv.best_score_))

elastic_pred= elasticreg_cv.predict(X_test)
rmse_elastic=np.sqrt(metrics.mean_squared_error(y_test,elastic_pred))
print("Root Mean Squared Error: {}".format(rmse_elastic))

### Using the best model for all samples
We got best $R^2$ result from *Random Forest Regressor*. Hence, we will use the model for all samples.

In [53]:

df5=pd.get_dummies(airbnb_data)
df5.shape

In [54]:
features=df5.drop(['price'],axis=1)
X=features
y=df5['price']

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3)
rfc=RandomForestRegressor()
rfc.fit(X_train,y_train)
y_pred= rfc.predict(X_test)

print("R^2: {}".format(rfc.score(X_test, y_test)))
rmse=np.sqrt(metrics.mean_squared_error(y_test,y_pred))
print("Root Mean Squared Error: {}".format(rmse))
cv_scores = cross_val_score(rfc,X,y,cv=5)

# Print the 5-fold cross-validation scores
print("CV Scores: {}".format(cv_scores))
print("Average 5-Fold CV Score: {}".format(np.mean(cv_scores)))